<a href="https://colab.research.google.com/github/PrajaktaPatil1/benign_malignant/blob/master/benign_malignant(with_GLCM_feature).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image

In [2]:
import os
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
import cv2
from skimage.feature import greycomatrix, greycoprops

In [4]:
from google.colab import files
uploaded = files.upload()

Saving archive (update1).zip to archive (update1).zip


In [5]:
import os
import zipfile

# Path to the zip file
zip_file_path = '/content/archive (update1).zip'

# Directory where you want to extract the files
extracted_dir = '/content/extracted_data'

# Ensure the directory exists
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# List the contents of the extracted directory
extracted_files = os.listdir(extracted_dir)
print("Contents of the extracted directory:")
for file_name in extracted_files:
    print(file_name)

# Destination directory to save the extracted files
save_dir = '/content/saved_files'

Contents of the extracted directory:
archive (4)


##Data Preprocessing and Feature Extraction

#Gray-Level Co-occurrence Matrix

In [6]:
# Step 1: Data Preprocessing

def extract_glcm_features(image):
    # Convert image to grayscale if it's not already
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    # Calculate GLCM
    distances = [1]  # distance between pixels
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # angles for co-occurrence matrix
    glcm = greycomatrix(gray, distances=distances, angles=angles, symmetric=True, normed=True)
    # Calculate GLCM properties
    contrast = greycoprops(glcm, 'contrast').ravel()
    dissimilarity = greycoprops(glcm, 'dissimilarity').ravel()
    homogeneity = greycoprops(glcm, 'homogeneity').ravel()
    energy = greycoprops(glcm, 'energy').ravel()
    correlation = greycoprops(glcm, 'correlation').ravel()
    return np.hstack([contrast, dissimilarity, homogeneity, energy, correlation])

def load_data(folder_path):
    X = []
    y = []
    for label, class_name in enumerate(['benign', 'malignant']):
        class_folder = os.path.join(folder_path, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            image = cv2.imread(img_path)
            features = extract_glcm_features(image)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)


In [7]:
# Step 2: Load and preprocess training data
train_folder = "/content/extracted_data/archive (4)/train"
X_train, y_train = load_data(train_folder)

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

In [8]:
# Step 3: Load and preprocess testing data
test_folder = "/content/extracted_data/archive (4)/test"
X_test, y_test = load_data(test_folder)

/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/usr/local/lib/python3.10/dist-packages/skimag

##Model Training

#Implementing Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
# Step 4: Train Logistic Regression model
lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
# Step 5: Evaluate model
y_pred_lr = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

Accuracy: 0.6556291390728477


In [13]:
from sklearn.metrics import accuracy_score, classification_report

In [14]:
accuracy_score(y_pred_lr,y_test)
print(classification_report(y_pred_lr,y_test))

              precision    recall  f1-score   support

           0       0.60      0.64      0.62        66
           1       0.70      0.67      0.69        85

    accuracy                           0.66       151
   macro avg       0.65      0.65      0.65       151
weighted avg       0.66      0.66      0.66       151



In [15]:
from sklearn.metrics import confusion_matrix

In [17]:
confusion_matrix(y_pred_lr,y_test)

array([[42, 24],
       [28, 57]])

Model selection with GridSearchCV

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
# Define hyperparameters grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}

In [20]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv=5)

In [21]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2']})

In [22]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

Best hyperparameters: {'C': 0.001, 'penalty': 'l2'}


In [24]:
# Initialize Logistic Regression with best hyperparameters
lr_tuned = LogisticRegression(**best_params)

In [25]:
# Fit the model with tuned hyperparameters
lr_tuned.fit(X_train, y_train)

LogisticRegression(C=0.001)

In [26]:
y_pred_lr = lr_tuned.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

Accuracy: 0.7019867549668874


In [27]:
from sklearn.metrics import accuracy_score, classification_report

In [28]:
accuracy_score(y_pred_lr,y_test)
print(classification_report(y_pred_lr,y_test))

              precision    recall  f1-score   support

           0       0.67      0.68      0.68        69
           1       0.73      0.72      0.72        82

    accuracy                           0.70       151
   macro avg       0.70      0.70      0.70       151
weighted avg       0.70      0.70      0.70       151



In [29]:
from sklearn.metrics import confusion_matrix

In [30]:
confusion_matrix(y_pred_lr,y_test)

array([[47, 22],
       [23, 59]])

#Implementing a Random Forest Classifier

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
rfc=RandomForestClassifier()

In [33]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [34]:
y_pred_rfc = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rfc)
print("Accuracy:", accuracy)

Accuracy: 0.8211920529801324


In [35]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [36]:
accuracy_score(y_pred_rfc,y_test)
print(classification_report(y_pred_rfc,y_test))

              precision    recall  f1-score   support

           0       0.80      0.81      0.81        69
           1       0.84      0.83      0.83        82

    accuracy                           0.82       151
   macro avg       0.82      0.82      0.82       151
weighted avg       0.82      0.82      0.82       151



In [37]:
confusion_matrix(y_pred_rfc,y_test)

array([[56, 13],
       [14, 68]])

Model Selection  with GridSearchCV

In [38]:
# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [39]:
# Create GridSearchCV object
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='accuracy')

In [40]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [41]:
# Print the best parameters found
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'bootstrap': True, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}


In [42]:
# Print the best estimator found
best_random_forest = grid_search.best_estimator_
print("\nBest Estimator:", best_random_forest)


Best Estimator: RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=10,
                       n_estimators=200)


In [43]:
y_pred_brf = best_random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_brf)
print("Accuracy:", accuracy)

Accuracy: 0.7086092715231788


In [44]:
from sklearn.metrics import accuracy_score, classification_report

In [45]:
accuracy_score(y_pred_brf,y_test)
print(classification_report(y_pred_brf,y_test))

              precision    recall  f1-score   support

           0       0.56      0.75      0.64        52
           1       0.84      0.69      0.76        99

    accuracy                           0.71       151
   macro avg       0.70      0.72      0.70       151
weighted avg       0.74      0.71      0.72       151



In [46]:
from sklearn.metrics import confusion_matrix

In [47]:
confusion_matrix(y_pred_brf,y_test)

array([[39, 13],
       [31, 68]])

#Implementing a KNN

In [48]:
from sklearn.neighbors import KNeighborsClassifier

In [49]:
knn=KNeighborsClassifier(n_neighbors=7)

In [50]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [51]:
y_pred_knn = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy)

Accuracy: 0.7284768211920529


In [52]:
accuracy_score(y_pred_knn,y_test)
print(classification_report(y_pred_knn,y_test))

              precision    recall  f1-score   support

           0       0.60      0.76      0.67        55
           1       0.84      0.71      0.77        96

    accuracy                           0.73       151
   macro avg       0.72      0.74      0.72       151
weighted avg       0.75      0.73      0.73       151



In [53]:
confusion_matrix(y_pred_knn,y_test)

array([[42, 13],
       [28, 68]])

#Implementing a Decision Tree

In [54]:
from sklearn.tree import DecisionTreeClassifier

In [55]:
dtc=DecisionTreeClassifier()

In [56]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [61]:
y_pred_dtc = dtc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_dtc)
print("Accuracy:", accuracy)

Accuracy: 0.8211920529801324


In [62]:
accuracy_score(y_pred_dtc,y_test)
print(classification_report(y_pred_dtc,y_test))

              precision    recall  f1-score   support

           0       0.81      0.80      0.81        71
           1       0.83      0.84      0.83        80

    accuracy                           0.82       151
   macro avg       0.82      0.82      0.82       151
weighted avg       0.82      0.82      0.82       151



In [63]:
confusion_matrix(y_pred_dtc,y_test)

array([[57, 14],
       [13, 67]])

model selection with gridsearchCV

In [64]:
# Define the hyperparameters to tune
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


Implementing a Naive Bayes classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb=GaussianNB()

In [ ]:
nb.fit(X_train, y_train)

GaussianNB()

In [ ]:
y_pred_nb = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_nb)
print("Accuracy:", accuracy)

Accuracy: 0.6235294117647059


In [ ]:
accuracy_score(y_pred_nb,y_test)
print(classification_report(y_pred_nb,y_test))

              precision    recall  f1-score   support

           0       0.66      0.63      0.65        93
           1       0.58      0.61      0.59        77

    accuracy                           0.62       170
   macro avg       0.62      0.62      0.62       170
weighted avg       0.63      0.62      0.62       170



In [ ]:
confusion_matrix(y_pred_nb,y_test)

array([[59, 34],
       [30, 47]])

Xgboost

In [ ]:
import xgboost as xgb

In [ ]:
# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

In [ ]:
# Train the classifier
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred_xgb)
print("Accuracy:", accuracy)

Accuracy: 0.8117647058823529


In [ ]:
accuracy_score(y_pred_xgb,y_test)
print(classification_report(y_pred_xgb,y_test))

              precision    recall  f1-score   support

           0       0.78      0.85      0.81        81
           1       0.85      0.78      0.81        89

    accuracy                           0.81       170
   macro avg       0.81      0.81      0.81       170
weighted avg       0.82      0.81      0.81       170

